In [1]:
!pip install -qU langchain-google-genai

In [2]:
from google.colab import userdata
API_KEY = userdata.get('GEMINI_API_KEY')

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model = "gemini-2.5-flash" , api_key = API_KEY)
response = model.invoke("what is your current knowledge cutoff date")
print(response.content)

My knowledge cutoff is **early 2023**. Therefore, I cannot provide you with any information about events or developments that have occurred since that time.

However, I can access real-time information through Google Search if I need to find current events or up-to-date data.


In [4]:
!pip install langchain langchain_community

In [5]:
from google.colab import userdata
TAVILY = userdata.get('tavily_API_KEY')

In [6]:
from langgraph.graph import MessageGraph , END
from langchain_core.messages import AIMessage , HumanMessage
from langchain_community.tools import TavilySearchResults

In [7]:

search = TavilySearchResults(tavily_api_key=TAVILY , max_results=2)

def planner_node(state):
  user_input = state[-1].content

  prompt=(
      "you are a smart planner the user will ask you the question, your job is to decide if you need to search online. if yes, the output should be the exact query that you need to be searched"
  )
  response = model.invoke([
      HumanMessage(content = prompt) ,
      HumanMessage(content = user_input)
  ]
  )

  return state + [AIMessage(content = response.content)]


def search_node(state):
  quary = state[-1].content
  search_res = search.invoke(quary)

  return state+ [AIMessage(content = str(search_res))]


def responder_node(state):
  question_given = state[0].content
  result_occur = state[-1].content
  prompt=(
      "you are a best responder take the given result and answer the user question"
  )
  response = model.invoke([
      HumanMessage(content = prompt) ,
      HumanMessage(content = f"user question : {question_given} and search result = {result_occur}") ,

  ]
  )

  return state + [AIMessage(content = response.content)]

graph = MessageGraph()
graph.add_node("Planner",planner_node)
graph.add_node("Search",search_node)
graph.add_node("Responder",responder_node)

graph.set_entry_point("Planner")

graph.add_edge("Planner","Search")
graph.add_edge("Search","Responder")
graph.add_edge("Responder",END)



app = graph.compile()
agent_response = app.invoke([
        HumanMessage(
            content="who is the current deputy chief minister of andhra pradesh"
        )
    ])
print(agent_response)













/tmp/ipython-input-177572214.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search = TavilySearchResults(tavily_api_key=TAVILY , max_results=2)
/tmp/ipython-input-177572214.py:40: LangGraphDeprecatedSinceV10: MessageGraph is deprecated in LangGraph v1.0.0, to be removed in v2.0.0. Please use StateGraph with a `messages` key instead. Deprecated in LangGraph V1.0 to be removed in V2.0.
  graph = MessageGraph()


[HumanMessage(content='who is the current deputy chief minister of andhra pradesh', additional_kwargs={}, response_metadata={}, id='fe409ac0-bd50-4e3f-9c1d-fef7c90a87e1'), AIMessage(content='current deputy chief minister of andhra pradesh', additional_kwargs={}, response_metadata={}, id='e433013a-f7d9-4a3e-9380-116ab0a39305', tool_calls=[], invalid_tool_calls=[]), AIMessage(content='[{\'title\': \'Pawan Kalyan - Wikipedia\', \'url\': \'https://en.wikipedia.org/wiki/Pawan_Kalyan\', \'content\': \'Konidela Pawan Kalyan (born Konidela Kalyan Babu; 2 September 1971) is an Indian politician, actor, philanthropist, and martial artist serving as the 11th Deputy Chief Minister of Andhra Pradesh since June 2024. He is also the Minister of Panchayat Raj, Rural Development and Rural Water Supply; Environment, Forest, Science and Technology in the Government of Andhra Pradesh as MLA "Member of the Legislative Assembly (India)") representing the Pithapuram constituency. He is the founding president

In [8]:
for i in agent_response:
  print(i.content)

who is the current deputy chief minister of andhra pradesh
current deputy chief minister of andhra pradesh
[{'title': 'Pawan Kalyan - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Pawan_Kalyan', 'content': 'Konidela Pawan Kalyan (born Konidela Kalyan Babu; 2 September 1971) is an Indian politician, actor, philanthropist, and martial artist serving as the 11th Deputy Chief Minister of Andhra Pradesh since June 2024. He is also the Minister of Panchayat Raj, Rural Development and Rural Water Supply; Environment, Forest, Science and Technology in the Government of Andhra Pradesh as MLA "Member of the Legislative Assembly (India)") representing the Pithapuram constituency. He is the founding president of the Janasena Party. [...] ### 2024–present: Deputy Chief Minister of Andhra Pradesh\n\nOn 12 June 2024, Pawan Kalyan was sworn in as a cabinet minister in the Government of Andhra Pradesh, and on 14 June 2024, he was announced as the deputy chief minister of Andhra Pradesh. He is also 

In [9]:
!pip install grandalf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.2 MB/s eta 0:00:00


In [10]:
print(app.get_graph().draw_ascii())

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
 +---------+   
 | Planner |   
 +---------+   
      *        
      *        
      *        
  +--------+   
  | Search |   
  +--------+   
      *        
      *        
      *        
+-----------+  
| Responder |  
+-----------+  
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   
